# Activity Investigation

This should be helpful when I just want to look closely at the results of a particular activity stream

In [ ]:
from strava.data.cache import Cache
from strava.data.strava_requests import *
from strava.plotting.strava_annual_plots import *
from strava.plotting.strava_stream_plots import *
from generate_plots import get_clean_activities, get_clean_stream
from strava.data.cache import Cache

import aiohttp
import asyncio
import copy
from datetime import date
import datetime
import json
import matplotlib.pyplot as plt
import numpy as np
import requests
import os
import pandas as pd
from plotnine import ggplot, aes
import plotnine as gg
import time
from typing import List
import urllib3
import PIL
from PIL import Image
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
gg.options.figure_size = (11, 8.5)
pt.dark()
pt.light()

pd.set_option('display.max_columns', 500)


%load_ext autoreload
%autoreload 2

In [3]:
segment_plot_path = '/Users/lucasnieuwenhout/Documents/Programming/Python/Projects/StravaPlotting/plots/segments/'

In [2]:
# Create a cache
stream_cache = Cache()
activity_cache = Cache(dir='/Users/lucasnieuwenhout/Documents/Programming/Python/Projects/StravaPlotting/activity_cache/')

In [3]:
# Get a list of activities
activities = get_clean_activities()
activities.tail()

Existing access token is valid.
Beginning request of activities...
-> Received page 1 with 200 activities.
-> Received page 2 with 200 activities.
-> Received page 3 with 200 activities.
-> Received page 4 with 74 activities.
-> Received page 5 with 0 activities.
...retrieved total of 674 activities.


/Users/lucasnieuwenhout/Documents/Programming/Python/Projects/StravaPlotting/src/generate_plots.py:150: FutureWarning: 'S' is deprecated and will be removed in a future version. Please use 's' instead of 'S'.


,id,name,distance,moving_time,elapsed_time,total_elevation_gain,elev_high,elev_low,sport_type,start_date_local,timezone,start_latlng,end_latlng,achievement_count,map.id,workout_type,average_speed,max_speed,average_heartrate,max_heartrate,map.summary_polyline,year,date_year_agnostic
668,3425015993,Afternoon Ride,8964.6,0 days 00:26:43,0 days 00:26:43,39.0,693.8,654.7,Ride,2020-04-26 15:07:20,(GMT-07:00) America/Edmonton,"[53.636688, -113.587822]","[53.636729, -113.587903]",0,a3425015993,10.0,20.1312,40.32,165.0,185.0,{mjfIhhjtTt@fAf@|@HREZUpAe@pB{@vC]z@a@r@k@x@c@...,2020,1970-04-26 15:07:20
669,3425016238,Lunch Ride,17516.8,0 days 00:48:11,0 days 00:48:11,71.2,693.8,653.7,Ride,2020-04-25 11:48:43,(GMT-07:00) America/Edmonton,"[53.636724, -113.587843]","[53.636742, -113.587806]",0,a3425016238,10.0,21.8124,50.04,178.0,194.0,knjfIrhjtT`AxAb@v@G|@UvA[pAw@fCy@hBg@t@g@l@u@n...,2020,1970-04-25 11:48:43
670,3425017204,Afternoon Ride,14517.0,0 days 00:47:28,0 days 00:47:28,64.1,693.8,653.7,Ride,2020-04-23 13:02:13,(GMT-07:00) America/Edmonton,"[53.636938, -113.587945]","[53.636747, -113.58785]",0,a3425017204,10.0,18.3492,36.72,175.4,198.0,qnjfIlhjtTfAvAl@bAEp@OdA]hBc@lBW~@]~@Yj@g@t@_@...,2020,1970-04-23 13:02:13
671,3425016478,Afternoon Ride,17605.8,0 days 00:57:41,0 days 00:57:41,69.9,693.8,653.7,Ride,2020-04-22 14:48:16,(GMT-07:00) America/Edmonton,"[53.636929, -113.587659]","[53.636694, -113.587956]",0,a3425016478,10.0,18.3132,39.96,176.6,194.0,wmjfIdhjtTx@pAZv@F^?XAb@EZUt@[r@qBpCw@rA_@b@s@...,2020,1970-04-22 14:48:16
673,3425016348,Afternoon Ride,7278.0,0 days 00:23:28,0 days 00:23:28,21.9,693.8,684.9,Ride,2020-04-16 13:57:58,(GMT-07:00) America/Edmonton,"[53.636894, -113.587782]","[53.636646, -113.587864]",0,a3425016348,10.0,18.6084,36.00,177.8,197.0,uojfIxhjtTvB|CjAnBbApBlA|CXdC^tEL`CB|ACtBKjBs@...,2020,1970-04-16 13:57:58


# Explore Activity API

What do I actually want to do?
- Plot my progress on segments of interest
- Plot how many of the 37 segments I have local legend on
- Get a list of the canonical segments and find my time vs. the best time?
- 

In [6]:
activities

,id,name,distance,moving_time,elapsed_time,total_elevation_gain,elev_high,elev_low,sport_type,start_date_local,timezone,start_latlng,end_latlng,achievement_count,map.id,workout_type,average_speed,max_speed,average_heartrate,max_heartrate,map.summary_polyline,year,date_year_agnostic
0,14411086862,Afternoon Ride,30259.6,0 days 01:14:01,0 days 01:17:07,387.3,100.6,4.4,Ride,2025-05-07 16:40:56,(GMT-08:00) America/Vancouver,"[49.05504, -122.774582]","[49.055064, -122.774611]",10,a14411086862,NaN,24.5304,47.664,161.7,181.0,a_ljHrsjlVFf@FcABBAt@H\\j@\hCN^Rx@LjBXZN`@DlAJ...,2025,1970-05-07 16:40:56
2,14380337660,Afternoon Ride,30228.5,0 days 01:17:27,0 days 01:20:01,387.3,100.6,4.4,Ride,2025-05-04 15:17:56,(GMT-08:00) America/Vancouver,"[49.055031, -122.774489]","[49.055048, -122.774634]",6,a14380337660,NaN,23.4180,52.416,164.0,187.0,s`ljHhtjlVPGNQN@NUFHB|@b@~@@r@z@nDHrAh@dAH`BNh...,2025,1970-05-04 15:17:56
3,14369761755,Afternoon Ride,27980.2,0 days 01:10:19,0 days 01:13:35,368.4,100.6,4.4,Ride,2025-05-03 15:38:33,(GMT-08:00) America/Vancouver,"[49.055028, -122.774625]","[49.055062, -122.774634]",6,a14369761755,NaN,23.8752,49.536,167.1,189.0,i~kjH~rjlVAz@BN^n@RdBr@vCB`ADRZ^N`@BfAJj@NTrC?...,2025,1970-05-03 15:38:33
4,14350214386,Afternoon Ride,27977.6,0 days 01:07:15,0 days 01:08:23,368.6,100.6,4.4,Ride,2025-05-01 16:59:56,(GMT-08:00) America/Vancouver,"[49.055076, -122.77454]","[49.055055, -122.774627]",8,a14350214386,NaN,24.9624,48.312,170.6,191.0,{`ljHntjlVH?b@]P?FMHC@LAt@b@|@PhBp@hCHxAZ\JTF\...,2025,1970-05-01 16:59:56
6,14307309734,Afternoon Ride,26564.7,0 days 01:06:34,0 days 01:10:29,351.3,100.6,4.4,Ride,2025-04-27 15:06:20,(GMT-08:00) America/Vancouver,"[49.055026, -122.774621]","[49.055073, -122.774604]",7,a14307309734,NaN,23.9436,51.696,165.4,185.0,o~kjH`sjlV@n@BZ`@x@RhBp@jCFjAX`@P`@BlAJj@HNNBb...,2025,1970-04-27 15:06:20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
641,3425015993,Afternoon Ride,8964.6,0 days 00:26:43,0 days 00:26:43,39.0,693.8,654.7,Ride,2020-04-26 15:07:20,(GMT-07:00) America/Edmonton,"[53.636688, -113.587822]","[53.636729, -113.587903]",0,a3425015993,10.0,20.1312,40.320,165.0,185.0,{mjfIhhjtTt@fAf@|@HREZUpAe@pB{@vC]z@a@r@k@x@c@...,2020,1970-04-26 15:07:20
642,3425016238,Lunch Ride,17516.8,0 days 00:48:11,0 days 00:48:11,71.2,693.8,653.7,Ride,2020-04-25 11:48:43,(GMT-07:00) America/Edmonton,"[53.636724, -113.587843]","[53.636742, -113.587806]",0,a3425016238,10.0,21.8124,50.040,178.0,194.0,knjfIrhjtT`AxAb@v@G|@UvA[pAw@fCy@hBg@t@g@l@u@n...,2020,1970-04-25 11:48:43
643,3425017204,Afternoon Ride,14517.0,0 days 00:47:28,0 days 00:47:28,64.1,693.8,653.7,Ride,2020-04-23 13:02:13,(GMT-07:00) America/Edmonton,"[53.636938, -113.587945]","[53.636747, -113.58785]",0,a3425017204,10.0,18.3492,36.720,175.4,198.0,qnjfIlhjtTfAvAl@bAEp@OdA]hBc@lBW~@]~@Yj@g@t@_@...,2020,1970-04-23 13:02:13
644,3425016478,Afternoon Ride,17605.8,0 days 00:57:41,0 days 00:57:41,69.9,693.8,653.7,Ride,2020-04-22 14:48:16,(GMT-07:00) America/Edmonton,"[53.636929, -113.587659]","[53.636694, -113.587956]",0,a3425016478,10.0,18.3132,39.960,176.6,194.0,wmjfIdhjtTx@pAZv@F^?XAb@EZUt@[r@qBpCw@rA_@b@s@...,2020,1970-04-22 14:48:16


# Activity Detail API

In [6]:
activ_id = activity_id_by_date(activities=activities, date=datetime.date(2025, 4, 18))
activity_detail = get_activity_detail(activities, activ_id, activity_cache, include_all_efforts=True)

In [7]:
def activity_detail_print_simple_summary(activity_detail):
    """Print simple summary of activity"""
    print(f"Date: \t\t{activity_detail['start_date_local'].split('T')[0]}")
    print(f"Name: \t\t{activity_detail['name']}")
    print(f"Distance: \t{activity_detail['distance'] / 1000:0.3f} km")
    print(f"Time: \t\t{activity_detail['moving_time'] // 3600}:{activity_detail['moving_time'] % 3600 // 60}:{activity_detail['moving_time'] % 60}")
    print(f"Elevation: \t{activity_detail['total_elevation_gain']} m")
    print(f"Achievements: \t{activity_detail['achievement_count']}")
    print(f"Average Speed: \t{activity_detail['average_speed'] / 1000 * 3600:0.2f} km/h")
    print(f"Average HR: \t{round(activity_detail['average_heartrate'])}")
    print(f"Segments: \t{len(activity_detail['segment_efforts'])}")

activity_detail_print_simple_summary(activity_detail)

Date: 		2025-04-18
Name: 		Afternoon Ride
Distance: 	28.507 km
Time: 		1:17:43
Elevation: 	352.0 m
Achievements: 	4
Average Speed: 	22.01 km/h
Average HR: 	168
Segments: 	37


In [8]:
activity_detail['segment_efforts'][0]

{'id': 3348481738355317272,
 'resource_state': 2,
 'name': '28th - it says bike route....',
 'activity': {'id': 14218149073,
  'visibility': 'everyone',
  'resource_state': 1},
 'athlete': {'id': 56778178, 'resource_state': 1},
 'elapsed_time': 111,
 'moving_time': 111,
 'start_date': '2025-04-18T23:43:15Z',
 'start_date_local': '2025-04-18T16:43:15Z',
 'distance': 734.2,
 'start_index': 123,
 'end_index': 234,
 'device_watts': False,
 'average_heartrate': 171.2,
 'max_heartrate': 176.0,
 'segment': {'id': 31582444,
  'resource_state': 2,
  'name': '28th - it says bike route....',
  'activity_type': 'Ride',
  'distance': 734.2,
  'average_grade': 2.0,
  'maximum_grade': 5.1,
  'elevation_high': 75.2,
  'elevation_low': 58.6,
  'start_latlng': [49.052882, -122.78008],
  'end_latlng': [49.052823, -122.790121],
  'elevation_profile': None,
  'elevation_profiles': None,
  'climb_category': 0,
  'city': 'Surrey',
  'state': 'British Columbia',
  'country': 'Canada',
  'private': False,
  'h

In [9]:
# [x for x in activity_detail['segment_efforts'][0]['achievements'].keys()]
activity_detail['segment_efforts'][0]['segment']['id']
activity_detail['segment_efforts'][0]['achievements']
{x['segment']['id']: x['achievements'] for x in get_activity_detail(activities, activ_id=14380337660, cache=activity_cache)['segment_efforts']}[36494958]#[3354257871155304832]#['achievements']
# 36494958

[{'type_id': 7,
  'type': 'segment_effort_count_leader',
  'rank': 1,
  'effort_count': 14}]

In [10]:
seg_dict = {x['segment']['name']: x['achievements'] for x in activity_detail['segment_efforts']}
for k, v in seg_dict.items():
    print(k)
    for ach in v:
        print(f'    {ach["type"]}: {ach["rank"]}')

28th - it says bike route....
    segment_effort_count_leader: 1
KG to 20 on 156
It smells richer in White Rock
    segment_effort_count_leader: 1
The Leafs Blow
    segment_effort_count_leader: 1
Marine Drive Rollers
white rockin'
Cruisin' East Beach - Westbound
marine kicker
Rollin' Restaurant Row
Hotties & Hot Rods
Escape from Beach Climb
Duprez to Magdalen
No E-Bikes allowed 
Marine Drive Gradual
Marine Drive Magdalen to Nichol
hill sprint marine drive
Tom's Sprint
    pr: 3
Ocean Park Road Sprint
Dude, where's my Bentley? - Reverse
Ocean Pk, 20 to 24
Pedestrians With Ice Cream - North
    segment_effort_count_leader: 1
Bikes & Bikinis
    segment_effort_count_leader: 1
Reverse Beaver
    segment_effort_count_leader: 1
Crescent Climb from Yacht Club to 128th
climb out of madness
    segment_effort_count_leader: 1
Cresent Rd E.B.Chevron to  Esso
    pr: 1
    segment_effort_count_leader: 1
Cresting Crescent
Crescent Rd. 
    pr: 1
    segment_effort_count_leader: 1
Crescent Rd Desce

In [11]:
# id(segment effort id), name, elapsed_time, moving_time, distance, average_heartrate, max_heartrate, segment
#  - name, distance, average_grade, maximum_grade, elevation_high, elevation_low, hazardous, pr_rank, achievements
#    - type, rank, effort_count

#   'resource_state': 2,
#   'name': '28th - it says bike route....',
#   'activity_type': 'Ride',
#   'distance': 734.2,
#   'average_grade': 2.0,
#   'maximum_grade': 5.1,
#   'elevation_high': 75.2,
#   'elevation_low': 58.6,
#   'start_latlng': [49.052882, -122.78008],
#   'end_latlng': [49.052823, -122.790121],
#   'elevation_profile': None,
#   'elevation_profiles': None,
#   'climb_category': 0,
#   'city': 'Surrey',
#   'state': 'British Columbia',
#   'country': 'Canada',
#   'private': False,
#   'hazardous': False,
#   'starred': False},
#  'pr_rank': None,
#  'achievements': [{'type_id': 7,
#    'type': 'segment_effort_count_leader',
#    'rank': 1,
#    'effort_count': 10}],

# Achievements
# Achievements is a list of dictinoaries holding various things
# - type_id = 7: 'type': 'segment_effort_count_leader', 'rank': 1, 'effort_count'
# - type_id = 3: 'type': 'pr', 'rank'
# - type_id = 2: 'type': 'overall', 'rank'

seg_df = []
for seg in activity_detail['segment_efforts']:
    segment_detail = [seg['id']]
    segment_detail.append(seg['segment']['id'])
    segment_detail.append(seg['name'])
    segment_detail.append(seg['distance'])
    segment_detail.append(seg['moving_time'])
    segment_detail.append(seg['average_heartrate'])
    segment_detail.append(seg['segment']['average_grade'])
    segment_detail.append(seg['segment']['hazardous'])
    segment_detail.append(seg['pr_rank'])
    segment_detail.append(seg['achievements'])
    if len(seg['achievements']) > 0:
        print(seg['achievements'])


    seg_df.append(segment_detail)
seg_df = pd.DataFrame(seg_df)
seg_df.columns = ['Effort ID', 'Segment ID', 'Name', 'Distance[m]', 'Moving Time[s]', 'Average HR', 'Average Grade', 'Hazardous', 'PR Rank', 'Achievements']

seg_df

[{'type_id': 7, 'type': 'segment_effort_count_leader', 'rank': 1, 'effort_count': 10}]
[{'type_id': 7, 'type': 'segment_effort_count_leader', 'rank': 1, 'effort_count': 10}]
[{'type_id': 7, 'type': 'segment_effort_count_leader', 'rank': 1, 'effort_count': 10}]
[{'type_id': 3, 'type': 'pr', 'rank': 3}]
[{'type_id': 7, 'type': 'segment_effort_count_leader', 'rank': 1, 'effort_count': 10}]
[{'type_id': 7, 'type': 'segment_effort_count_leader', 'rank': 1, 'effort_count': 10}]
[{'type_id': 7, 'type': 'segment_effort_count_leader', 'rank': 1, 'effort_count': 10}]
[{'type_id': 7, 'type': 'segment_effort_count_leader', 'rank': 1, 'effort_count': 10}]
[{'type_id': 3, 'type': 'pr', 'rank': 1}, {'type_id': 7, 'type': 'segment_effort_count_leader', 'rank': 1, 'effort_count': 10}]
[{'type_id': 3, 'type': 'pr', 'rank': 1}, {'type_id': 7, 'type': 'segment_effort_count_leader', 'rank': 1, 'effort_count': 10}]
[{'type_id': 7, 'type': 'segment_effort_count_leader', 'rank': 1, 'effort_count': 10}]
[{'typ

,Effort ID,Segment ID,Name,Distance[m],Moving Time[s],Average HR,Average Grade,Hazardous,PR Rank,Achievements
0,3348481738355317272,31582444,28th - it says bike route....,734.200,111,171.2,2.0,False,NaN,"[{'type_id': 7, 'type': 'segment_effort_count_..."
1,3348481738356708888,25667954,KG to 20 on 156,474.000,70,176.1,-0.1,False,NaN,[]
2,3348481738355444248,10267456,It smells richer in White Rock,1729.200,231,180.5,0.1,False,NaN,"[{'type_id': 7, 'type': 'segment_effort_count_..."
3,3348481738357081624,20245971,The Leafs Blow,374.900,50,145.0,-4.9,False,NaN,"[{'type_id': 7, 'type': 'segment_effort_count_..."
4,3348481738356379160,1145071,Marine Drive Rollers,1888.500,585,149.2,-0.2,True,NaN,[]
5,3348481738357499416,3935337,white rockin',1847.700,574,149.4,-0.3,True,NaN,[]
6,3348481738357747224,19129655,Cruisin' East Beach - Westbound,404.600,150,140.2,0.4,False,NaN,[]
7,3348481738356374040,16233163,marine kicker,317.200,49,181.9,6.4,False,NaN,[]
8,3348481738355226136,19059308,Rollin' Restaurant Row,616.400,261,146.5,-1.1,False,NaN,[]
9,3348481738358563352,19628715,Hotties & Hot Rods,486.600,69,153.0,0.7,False,NaN,[]
